In [4]:
# simple feed forward network with ReLU example
import numpy as np

if __name__=='__main__':
    # parameters
    inp_size = 10 # input size
    etha = 0.1 # learning rate
    niter = 100

    # input
    x = np.zeros((1, inp_size)) # input
    x = [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]

    # model parameters
    W1 = np.random.randn(inp_size, inp_size)*0.01 # input to hidden
    W2 = np.random.randn(inp_size, inp_size)*0.01 # hidden to output
    b1 = np.zeros((1, inp_size)) # inp-hidden bias
    b2 = np.zeros((1, inp_size)) # hidden-out bias
    
    for ictr in range(niter):

        # forward pass
        h1 = np.dot(x, W1) + b1
        h1 = np.maximum(h1, 0, h1) # ReLU
        o2 = np.dot(h1, W2) + b2
        #print(o2)

        # backward pass
        y = [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]
        h1 = np.dot(x, W1) + b1
        dW1 = - etha * (o2 - y) * np.maximum(h1, 0, h1)
        dW2 = dW1 * ((h1 > 0) * 1.) * x
        
        W1 += dW1
        W2 += dW2
        
    print(dW1)
    print(dW2)
    # forward pass
    h1 = np.dot(x, W1) + b1
    h1 = np.maximum(h1, 0, h1) # ReLU
    o2 = np.dot(h1, W2) + b2
    print(o2)

[[-0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
   1.81831121e-07 -0.00000000e+00  0.00000000e+00  1.12753895e-08
  -0.00000000e+00  0.00000000e+00]]
[[-0.  0.  0. -0.  0. -0.  0.  0. -0.  0.]]
[[ 8.04376336e-05 -5.65068638e-05 -7.77747678e-05  1.29369767e-04
  -1.76518582e-04  7.90246546e-05 -2.17529744e-04 -1.93975479e-05
   4.26013069e-05 -6.47007077e-05]]


In [36]:
# key-value memory network

import numpy as np
from gensim import corpora
import math

def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

def lj(j, J, k, d): 
    return (1-j/J)-(k/d)*(1-2*j/J)

def embed(facts_raw, query_raw, uniq):
    facts = facts_raw.lower().split('.')
    facts.append(query_raw.lower())
    #senttoken = [ [word for word in sentence.lower().split(' ') if word not in stoplist] for sentence in document ]
    senttoken = []
    for idoc in range(len(facts)):
        thissen = facts[idoc].lower().split(' ')
        tokendoc = []
        for idx, thiswrd in enumerate(thissen):
            if uniq:
                tokendoc.append(thiswrd + '_' + str(idoc)) # for seaparate encoding (Jonathan Huis blog)
            else:
                tokendoc.append(thiswrd)
        senttoken.append(tokendoc)
    dictionary = corpora.Dictionary(senttoken)
    facts.pop(len(facts)-1) # query at the end of document
    d_embed = len(dictionary) # embedding dimension
    return(senttoken, facts, dictionary, d_embed)

if __name__=="__main__":
    # load input data
    #data = open('input1.txt', 'r').read() # input text file: sentences separated by .

    # hyperparameters
    lr = 1.3 # learning rate
    
    # input to memory embedding mi = A * xi + tA
        #stoplist = set('for a of the and to in'.split())
    stoplist = []
    facts_raw = 'Mary moved to the bathroom. John went to the hallway.'
    
    # create key-value pairs
    query_raw = 'Where is Mary'
    
    senttoken, facts, dictionary, d_embed = embed(facts_raw, query_raw, False)
    voc = d_embed
    n_memories = len(facts)
    
    print(dictionary.token2id)

    # initiate weigth matrices
    A = np.random.randn(d_embed, voc)*0.01 # input to memory embedding = key-value and query embeddings
    B = np.random.randn(d_embed, voc)*0.01 # query embedding = for y-embeddings
    R1 = np.random.randn(voc, d_embed)*0.01 # final weight matrix = R1 for hop-to-hop query updates
    
    # memory for Adagrad
    mA = np.zeros_like(A)
    mB = np.zeros_like(B)
    mR1 = np.zeros_like(R1)

    phi_k = np.zeros((n_memories, voc)) # keys
    phi_v = np.zeros((n_memories, voc)) # values
    phi_y = np.zeros(voc) # candidates
    for i in range(n_memories):
        for j in range(len(senttoken[i])):
            phi_k[i][j] = dictionary.token2id[senttoken[i][j]]
            phi_v[i][j] = dictionary.token2id[senttoken[i][j]] # for now the same as keys
    
    for i in range(len(dictionary)):
        phi_y[i] = i # each word in the vocabulary is a potential candidate
        
    # todo: key hashing (inverted index)
         
    for iterctr in range(500):

        # forward pass
        
        # embedding key = A * phi_k
        key = np.zeros((n_memories, d_embed))
        for i in range(n_memories):
            key[i] = np.dot(A, phi_k[i].T) # simple embedding
            #m[i][j] = lj(j,len(document[i]),j,d_embed) * A[i][j] * x[i][j] # with positional encoding

        # embedding query q = A * phi_x
        phi_x = np.zeros(voc)
        qj1 = np.zeros(d_embed)
        thissent = senttoken[3] # test query: where is mary
        for j in range(len(thissent)):
            phi_x[j] = dictionary.token2id[thissent[j]]
        qj1 = np.dot(A, phi_x)
        
        # key addressing

        # read head of the Neural Turing Machine
        # match of query with key p = softmax(qj1 * A * key) for all i
        pk = np.zeros((n_memories, d_embed))
        pk = softmax(np.dot(qj1, key.T))

        # output corresponding to input xi: ci = C * xi
        val = np.zeros((n_memories, d_embed))
        for i in range(n_memories):
            val[i] = np.dot(A, phi_v[i].T)
            
        # value reading

        # internal state of the controller
        # response vector from memory o = sum pi * ci
        o = np.zeros(d_embed)
        o = np.dot(pk.T, val)
        
        # 2nd hop
        qj1 = np.dot(R1, (qj1 + o))
        #pk = softmax(np.dot(qj1, np.dot(A, key.T)))
        pk = softmax(np.dot(qj1, key.T))
        o = np.dot(pk.T, val)
        qj1 = np.dot(R1, (qj1 + o))
        
        # 3rd hop
        qj1 = np.dot(R1, (qj1 + o))
        #pk = softmax(np.dot(qj1, np.dot(A, key.T)))
        pk = softmax(np.dot(qj1, key.T))
        o = np.dot(pk.T, val)
        qj1 = np.dot(R1, (qj1 + o))

        # predicted label a = softmax( R1 * (o + u) * B * phi_y)
        #a_predict = softmax(np.dot(R1, (qj1 + o)))
        a_predict = softmax(qj1 * np.dot(B, phi_y))
        #a_predict = softmax(qj1 * np.dot(A, phi_y)) # only using A
        #print(a_predict)

        # backpropagation

        dA = np.zeros_like(A)
        dB = np.zeros_like(B)
        dR1 = np.zeros_like(R1)

        truth = np.zeros(voc)
        truth[0] = 1. # answer (bathroom)
        dy = a_predict - truth
        #print(dy)
        #print('V: %d' % (voc))
        #print('d: %d', (d_embed))
        ABunit = np.pad(np.identity(voc), ((0,d_embed-voc),(0,0)), 'constant', constant_values=(0))
        R1unit = np.pad(np.identity(voc), ((0,0), (0,d_embed-voc)), 'constant', constant_values=(0))

        # dA = dy a_predict * (1-a_predict) R1 (phi_x + sumi ( p[i] (1-p[i]) ( phi_x A phi_K + A phi_x phi_K ) A phi_V + pki phi_V)
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += pk[i]*(1.-pk[i]) * np.dot(np.dot(qj1, np.dot(ABunit, phi_k[i].T)), val[i])
        dEAtmp = R1 * dEAtmp
        dA = (np.dot(dy, a_predict * (1-a_predict)) * dEAtmp).T
        #print(dA)

        # dB = dy a_predict * (1-a_predict) q phi_y
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += pk[i]*(1.-pk[i])*np.dot(np.dot(ABunit, qj1), phi_y[i])
        dEAtmp = R1 * dEAtmp
        dB = (np.dot(dy, a_predict*(1-a_predict)) * dEAtmp).T
        #print(dB)

        # dR1 = dy a_predict * (1-a_predict) (q + o) B phi_y
        #print(np.shape(np.dot((qj1 + o), np.dot(B, phi_y.T))))
        dR1 = np.dot(dy, a_predict*(1-a_predict)) * R1unit * np.dot((qj1 + o), np.dot(B, phi_y.T))
        #print(dR1)   

        # maybe clip ?
        #for dweights in [dA,dB,dR1]:
            #np.clip(dweights, -5., 5., out=dweights) # exploding gradients (but seems well-behaved enough)

        # update weights with Adagrad
        for weights, dweights, memwghts in zip([A,B,R1], [dA,dB,dR1], [mA,mB,mR1]):
            #memwghts += dweights * dweights
            #weights += -lr * dweights / np.sqrt(memwghts + 1.e-8)
            weights += -lr * dweights

        #print(A)
    print(a_predict)
    #print(np.argmax(a_predict))
    print(dictionary[np.argmax(a_predict)])

{'bathroom': 0, 'mary': 1, 'moved': 2, 'the': 3, 'to': 4, '': 5, 'hallway': 6, 'john': 7, 'went': 8, 'is': 9, 'where': 10}
[0.09090772 0.09089124 0.09092105 0.09091639 0.09092264 0.0908501
 0.09094262 0.09092464 0.09092496 0.09084815 0.09095049]
where


In [3]:
# prepare text file as corpus (lower case, remove stopwords) and tokenize
import os
import re # regex
from gensim.parsing.preprocessing import preprocess_string

def read1k():
    return f.read(1024)

def process_data(chunk, text):
    text.append(str(chunk)) # 'utf8' codec can't decode byte 0xc3

def rmsword(corpus, stopwords): # remove stopwords from corpus
    i = 0
    for elem in corpus:
        for sword in stopwords:
            if elem == sword:
                while True:
                    try:
                        corpus.remove(elem) # this throws an error if elem not in corpus (might have been removed already)
                        i += 1
                    except:
                        break
    return i # returns number of stopwords removed

def chunks(l, n): # Yield successive n-sized chunks from list l
    for i in range(0, len(l), n):
        yield l[i:i + n] # returns a generator

def chunksep(l, s): # Yield successive chunks from list l separated by s
    g = []
    for el in l:
        if el == s:
            yield g
            g = []
        g.append(el)
    yield g

if __name__=="__main__":
    text = 'this is a test with a lot of ambiguity.'
    #corpus = preprocess_string(' '.join(text)) # requires string
    corpus = preprocess_string(text)
    print(corpus)

['test', 'lot', 'ambigu']


In [1]:
# key-value memories: first embedding tests
from gensim import corpora

def docstr2lst(text): # text string with sentences separated by ., returns list of lists of sentences
    memraw = []
    for mem in text.split('.'):
        memraw.append(mem.split(' '))
    return memraw

def text2bow(memraw, memdict): # raw text to bow (maps each token to its id, takes a list of lists of sentence words)
    membow = []
    for mem in memraw:
        memline = []
        for memw in mem:
            memline.append(memdict.token2id[memw])
        membow.append(memline)
    return membow

def windowlvlstr(text, lenW): # return key=entire window, value=center word, string version
    if lenW % 2 == 0 or len(text) <= lenW:
        return ([],[])
    textl = text.split(' ')
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(textl)-lenW2):
        thiskey = []
        retvals.append(textl[ictr])
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(textl[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

def windowlvl(text, lenW): # return key=entire window, value=center word, BOW version (text a list of ids)
    if lenW % 2 == 0 or len(text) <= lenW:
        return ([],[])
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(text)-lenW2):
        thiskey = []
        retvals.append(text[ictr]) # center encoding: would need to add a different dict here
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(text[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

# for window+center encoding: add a step for the center word:
# translate the center back to the original with dic 0, then with dic 1 to the center encoding
def windowclvl(text, lenW, memdict, cdict): # return key=entire window, value=center word, BOW version with center
    if lenW % 2 == 0 or len(text) <= lenW:
        return ([],[])
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(text)-lenW2):
        thiskey = []
        retvals.append(cdict.token2id[memdict[text[ictr]]]) # center encoding: different dict for ctr word
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(text[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

if __name__=="__main__":
    text = 'this is a test with a lot of ambiguity. docs are split by periods.'
    memraw = docstr2lst(text)
    memdict = corpora.Dictionary(memraw)
    #print(memdict.token2id)
    #print(memdict[1])
    #print(windowlvlstr(text, 3))
    #print(windowlvlstr(text, 5))
    # text to bow
    membow = text2bow(memraw, memdict)
    print(membow)
    print(windowlvl(membow[0], 3))
    print(windowclvl(membow[0], 3, memdict, memdict)) # test with the same for now - should be the same result as windowlvl

[[6, 2, 0, 5, 7, 0, 3, 4, 1], [8, 11, 9, 13, 10, 12], [8]]
([[6, 2, 0], [2, 0, 5], [0, 5, 7], [5, 7, 0], [7, 0, 3], [0, 3, 4], [3, 4, 1]], [2, 0, 5, 7, 0, 3, 4])
([[6, 2, 0], [2, 0, 5], [0, 5, 7], [5, 7, 0], [7, 0, 3], [0, 3, 4], [3, 4, 1]], [2, 0, 5, 7, 0, 3, 4])


In [9]:
'''
This implements: http://en.wikipedia.org/wiki/Inverted_index of 28/07/10
'''
 
from pprint import pprint as pp
from glob import glob
try: reduce
except: from functools import reduce
try:    raw_input
except: raw_input = input
 
 
def parsetexts(fileglob='InvertedIndex.txt'):
    texts, words = {}, set()
    for txtfile in glob(fileglob):
        with open(txtfile, 'r') as f:
            txt = f.read().split()
            words |= set(txt)
            texts[txtfile.split('\\')[-1]] = txt
    return texts, words
 
def termsearch(terms): # Searches simple inverted index
    return reduce(set.intersection,
                  (invindex[term] for term in terms),
                  set(texts.keys()))
 
texts, words = parsetexts()
print('\nTexts')
pp(texts)
print('\nWords')
pp(sorted(words))

invindex = {word:set(txt
                        for txt, wrds in texts.items() if word in wrds)
            for word in words}
print('\nInverted Index')
pp({k:sorted(v) for k,v in invindex.items()})
 
terms = ["what", "is", "it"]
print('\nTerm Search for: ' + repr(terms))
pp(sorted(termsearch(terms)))


Texts
{'InvertedIndex.txt': ['it',
                       'is',
                       'what',
                       'it',
                       'is',
                       'what',
                       'is',
                       'it',
                       'it',
                       'is',
                       'a',
                       'banana']}

Words
['a', 'banana', 'is', 'it', 'what']

Inverted Index
{'a': ['InvertedIndex.txt'],
 'banana': ['InvertedIndex.txt'],
 'is': ['InvertedIndex.txt'],
 'it': ['InvertedIndex.txt'],
 'what': ['InvertedIndex.txt']}

Term Search for: ['what', 'is', 'it']
['InvertedIndex.txt']


In [11]:
from collections import Counter
 
def termsearch(terms): # Searches full inverted index
    if not set(terms).issubset(words):
        return set()
    return reduce(set.intersection,
                  (set(x[0] for x in txtindx)
                   for term, txtindx in finvindex.items()
                   if term in terms),
                  set(texts.keys()) )
 
def phrasesearch(phrase):
    wordsinphrase = phrase.strip().strip('"').split()
    if not set(wordsinphrase).issubset(words):
        return set()
    #firstword, *otherwords = wordsinphrase # Only Python 3
    firstword, otherwords = wordsinphrase[0], wordsinphrase[1:]
    found = []
    for txt in termsearch(wordsinphrase):
        # Possible text files
        for firstindx in (indx for t,indx in finvindex[firstword]
                          if t == txt):
            # Over all positions of the first word of the phrase in this txt
            if all( (txt, firstindx+1 + otherindx) in finvindex[otherword]
                    for otherindx, otherword in enumerate(otherwords) ):
                found.append(txt)
    return found
 

finvindex = {word:set((txt, wrdindx)
                      for txt, wrds in texts.items()
                      for wrdindx in (i for i,w in enumerate(wrds) if word==w)
                      if word in wrds)
             for word in words}
print('\nFull Inverted Index')
pp({k:sorted(v) for k,v in finvindex.items()})
 
print('\nTerm Search on full inverted index for: ' + repr(terms))
pp(sorted(termsearch(terms)))
 
phrase = '"what is it"'
print('\nPhrase Search for: ' + phrase)
print(phrasesearch(phrase))
 
# Show multiple match capability
phrase = '"it is"'
print('\nPhrase Search for: ' + phrase)
ans = phrasesearch(phrase)
print(ans)
ans = Counter(ans)
print('  The phrase is found most commonly in text: ' + repr(ans.most_common(1)[0][0]))


Full Inverted Index
{'a': [('InvertedIndex.txt', 10)],
 'banana': [('InvertedIndex.txt', 11)],
 'is': [('InvertedIndex.txt', 1),
        ('InvertedIndex.txt', 4),
        ('InvertedIndex.txt', 6),
        ('InvertedIndex.txt', 9)],
 'it': [('InvertedIndex.txt', 0),
        ('InvertedIndex.txt', 3),
        ('InvertedIndex.txt', 7),
        ('InvertedIndex.txt', 8)],
 'what': [('InvertedIndex.txt', 2), ('InvertedIndex.txt', 5)]}

Term Search on full inverted index for: ['what', 'is', 'it']
['InvertedIndex.txt']

Phrase Search for: "what is it"
['InvertedIndex.txt']

Phrase Search for: "it is"
['InvertedIndex.txt', 'InvertedIndex.txt', 'InvertedIndex.txt']
  The phrase is found most commonly in text: 'InvertedIndex.txt'


In [ ]:
# -*- coding: utf-8 -*-

import pickle
from nltk.tokenize import word_tokenize
import numpy as np
import functools
from itertools import chain
import copy

def save_pickle(d, path):
    print('save pickle to', path)
    with open(path, mode='wb') as f:
        pickle.dump(d, f)

def load_pickle(path):
    print('load', path)
    with open(path, mode='rb') as f:
        return pickle.load(f)

def lower_list(word_list):
    return [w.lower() for w in word_list]

def load_entities(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        entities = [e.lower().rstrip() for e in lines]
        return list(set(entities))

def find_ngrams(token_dict, text, n):
    """ See: https://github.com/facebookresearch/ParlAI/blob/master/parlai/core/dict.py#L31
        token_dict:  {'hello world', 'ol boy'}
        text: ['hello', 'world', 'buddy', 'ol', 'boy']
        n: max n of n-gram
        ret: ['hello world', 'buddy', 'ol boy']
    """
    """Breaks text into ngrams that appear in ``token_dict``."""
    # base case
    if n <= 1:
        return text
    # tokens committed to output
    saved_tokens = []
    # tokens remaining to be searched in sentence
    search_tokens = text[:]
    # tokens stored until next ngram found
    next_search = []
    while len(search_tokens) >= n:
        ngram = ' '.join(search_tokens[:n])
        if ngram in token_dict:
            # first, search previous unmatched words for smaller ngrams
            sub_n = min(len(next_search), n - 1)
            saved_tokens.extend(find_ngrams(token_dict, next_search, sub_n))
            next_search.clear()
            # then add this ngram
            saved_tokens.append(ngram)
            # then pop this ngram from the remaining words to search
            search_tokens = search_tokens[n:]
        else:
            next_search.append(search_tokens.pop(0))
    remainder = next_search + search_tokens
    sub_n = min(len(remainder), n - 1)
    saved_tokens.extend(find_ngrams(token_dict, remainder, sub_n))
    return saved_tokens

def load_task(fpath):
    print('load', fpath)
    with open (fpath, encoding='utf-8') as f:
        lines = f.readlines()
        data = []
        for i, l in enumerate(lines):
            l = l.rstrip()
            turn, left = l.split(' ', 1)
            
            if '\t' in left: # question
                q, a = left.split('\t', 1)
                q = q.split('?', 1)[1] # use split by n-gram
                q = q.split(' 1:')[1:]
                q = lower_list(q)
            
                a = a.split(', ') # may contain several labels
                a = lower_list(a)
                data.append((q, a))
    return data
    # data_q = load_questions(q_fpath)
    # data_a = load_answers(a_fpath)
    # return [(q, a) for q, a in zip(data_q, data_a)]

# def load_task(fpath, token_dict=None, max_token_length=None):
#     # print('load', fpath)
#     # with open (fpath, encoding='utf-8') as f:
#     #     lines = f.readlines()
#     #     data, story = [], []
#     #     for i, l in enumerate(lines):
#     #         if i % 2000 == 0: print(i, '/', len(lines))
#     #         l = l.rstrip()
#     #         turn, left = l.split(' ', 1)
            
#     #         if turn == '1': # new story
#     #             story = []

#     #         if '\t' in left: # question
#     #             q, a = left.split('\t', 1)
#     #             if q[-1] == '?':
#     #                 q = q[:-1]
#     #             # q = word_tokenize(q)
#     #             q = q.split(' ')
#     #             q = lower_list(q)
#     #             if token_dict and max_token_length:
#     #                 q = find_ngrams(token_dict, q, max_token_length)

#     #             if '\t' in a:
#     #                 a = a.split('\t')[0] # discard reward
#     #             a = a.split('|') # may contain several labels
#     #             a = lower_list(a)

#     #             substory = [x for x in story if x]

#     #             data.append((substory, q, a))
#     #             story.append('')
#     #         else: # normal sentence
#     #             # s = word_tokenize(left)
#     #             s = left.split(' ')
#     #             if s[-1] == '.':
#     #                 s = s[:-1]
#     #             s = lower_list(s)
#     #             story.append(s)

#     # return data

def vectorize(data, w2i, query_maxlen, w2i_label, use_multi_label=False):
    Q, A = [], []
    for question, answer in data:
        # Vectroize question
        q = [w2i[w] for w in question if w in w2i]
        q = q[:query_maxlen]
        q_pad_len = max(0, query_maxlen - len(q))
        q += [0] * q_pad_len

#         y = np.zeros(len(w2i_label))
#         y[w2i_label[answer[0]]] = 1
        y = np.zeros(len(w2i_label))
        if use_multi_label:
            for a in answer:
                y[w2i_label[a]] = 1
        else:
            y[w2i_label[answer[0]]] = 1

        Q.append(q)
        A.append(y)
    
    Q = np.array(Q, dtype=np.uint32)
    A = np.array(A, dtype='byte')

    return Q, A

def load_kv_pairs(path, token_dict, max_token_length, is_save_pickle=False):
    """load key-value paris from KB"""
    # rel = ['directed_by', 'written_by', 'starred_actors', 'release_year', 'has_genre', 'has_tags', 'has_plot', 'in_language'] # TODO hard code
    rel = ['directed_by', 'written_by', 'starred_actors', 'release_year', 'has_genre', 'has_tags', 'in_language'] # TODO hard code, not use has_plot tag
    kv_pairs = []
    with open(path, 'r') as f:
        lines = f.readlines()
        for i, l in enumerate(lines):
            if i % 5000 == 0: print('load_kv_pairs:', i, '/', len(lines))
            if l == '\n': continue
            turn, left = l.rstrip().split(' ', 1)
            for r in rel:
                if r in left:
                    k, v = [], []
                    tmp = left.split(r)
                    lhs = tmp[0].rstrip().lower()
                    k.append(lhs)
                    k.append(r)
                    rhs = tmp[1].strip().lower()
                    vals = rhs.split(', ')
                    for v in vals:
                        kv_pairs.append((k, [v]))

                        # double KB by considering reversed relation. see 3.2
                        k_r = [v, '!'+r]
                        v_r = [lhs]
                        kv_pairs.append((k_r, v_r))

                    break

    if is_save_pickle:
        save_pickle(kv_pairs, 'pickle/mov_kv_pairs.pickle')

    return kv_pairs

def vectorize_kv(data, max_mem_len, max_mem_size, w2i):
    all_vec_list = []
    for i, kv_list in enumerate(data):
        if i % 5000 == 0: print('vectorize_kv:', i, '/', len(data))
        vec_list = []
        for kv in kv_list[:max_mem_len+100]: #TODO: +100 for unknown entity in w2i
            vec = [w2i[e] for e in kv if e in w2i]
            # vec = [w2i[e] for e in kv]
            vec = vec[:max_mem_len]
            mem_pad_len = max(0, max_mem_len - len(vec))
            vec = vec + [0] * mem_pad_len
            vec_list.append(vec)
        vec_list = vec_list[:max_mem_size]
        mem_pad_size = max(0, max_mem_size - len(vec_list))
        for _ in range(mem_pad_size):
            vec_list.append([0] * max_mem_len)
        all_vec_list.append(vec_list)

    return np.array(all_vec_list, dtype=np.uint32)

def load_kv_dataset(data, kv_pairs, stopwords):
    print('---',len(data), len(kv_pairs))
    data_k, data_v = [], []
    for i, (q, _) in enumerate(data):
        if i%100 == 0: print('load_kv_dataset:', i, '/', len(data))
        k_list, v_list = [], []
        for w in q:
            if w not in stopwords:
                for kv_ind, (k, v) in enumerate(kv_pairs):
                    if w in (k): # the key shares at least one word with question with F<1000
                        k_list.append(k)
                        v_list.append(v)
        if len(k_list) == 0:
            print('==================no kv!')
            print(q)
        if len(k_list) > 100:
            print('==================too many kv! > 100')
            print(q)
            print(len(k_list))
        data_k.append(k_list)
        data_v.append(v_list)
        
    return data_k, data_v

def get_stop_words(data, freq, token_dict, max_token_length, is_save_pickle):
    # train_data = load_task('./data/movie_dialog_dataset/task1_qa/task1_qa_pipe_train.txt')
    # test_data = load_task('./data/movie_dialog_dataset/task1_qa/task1_qa_pipe_test.txt')
    # dev_data = load_task('./data/movie_dialog_dataset/task1_qa/task1_qa_pipe_dev.txt')
    # data = train_data + test_data + dev_data
    bow = {}
    for i, (q, _) in enumerate(data):
        if i % 2000 == 0: print(i, '/', len(data))
        for qq in q:
            q_tokens = find_ngrams(token_dict, qq.split(' '), max_token_length)
            for w in q_tokens:
                if w not in bow:
                    bow[w] = 0
                else:
                    bow[w] += 1

    stopwords = [k for k, v in bow.items() if v >= freq]
    if is_save_pickle:
        save_pickle(stopwords, 'pickle/mov_stopwords.pickle')

    return stopwords

def filter_data(data, data_k, data_v, kv_min, kv_max):
    indices = []
    for i, k in enumerate(data_k):
        if len(k) > kv_min and len(k) <= kv_max:
            indices.append(i)
    data = [data[i] for i in indices]
    data_k = [data_k[i] for i in indices]
    data_v = [data_v[i] for i in indices]
    return data, data_k, data_v

if __name__ == '__main__':
    # --- entities
    entities = load_pickle('pickle/mov_entities.pickle')
    # entities = load_entities('./data/movieqa/knowledge_source/entities.txt')
    # save_pickle(entities, 'mov_entities.pickle')
    # max_entity_length = max(map(len, (e.split(' ') for e in entities))) # for searching n-gram
    # vocab = load_pickle('pickle/mov_vocab.pickle')

    # --- movie-qa train/test dataset
    train_data = load_task('./data/WikiMovies/train_1.txt')
    test_data = load_task('./data/WikiMovies/test_1.txt')
    dev_data = load_task('./data/WikiMovies/dev_1.txt')
    save_pickle(train_data, 'pickle/mov_task1_qa_pipe_train.pickle')
    save_pickle(test_data, 'pickle/mov_task1_qa_pipe_test.pickle')
    save_pickle(dev_data, 'pickle/mov_task1_qa_pipe_dev.pickle')
    # train_data = load_pickle('pickle/mov_task1_qa_pipe_train.pickle')
    # test_data = load_pickle('pickle/mov_task1_qa_pipe_test.pickle')
    # dev_data = load_pickle('pickle/mov_task1_qa_pipe_dev.pickle')
    print(len(train_data))

    # -- update vocab and w2i/i2w
    vocab = set(entities 
                + ['directed_by', 'written_by', 'starred_actors', 'release_year', 'has_genre', 'has_tags', 'in_language'] 
                + ['!directed_by', '!written_by', '!starred_actors', '!release_year', '!has_genre', '!has_tags', '!in_language'] )
    for q, answer in train_data + test_data + dev_data:
        vocab |= set(q + answer)
    vocab = sorted(list(vocab))
    save_pickle(vocab, 'pickle/mov_vocab.pickle')
    w2i = dict((c, i) for i, c in enumerate(vocab, 1))
    i2w = dict((i, c) for i, c in enumerate(vocab, 1))
    save_pickle(w2i, 'pickle/mov_w2i.pickle')
    save_pickle(i2w, 'pickle/mov_i2w.pickle')
    # vocab = load_pickle('pickle/mov_vocab.pickle')
    
    # generate kv_pairs
    # kv_pairs = load_kv_pairs('./data/movieqa/knowledge_source/wiki_entities/wiki_entities_kb.txt', entities,  100, True)
    # kv_pairs = load_pickle('pickle/mov_kv_pairs.pickle')
    # vec_kv_pairs = vectorize_kv_pairs(kv_pairs, 10, 30, entities)
    
    # generate stopwords
    # stopwords = get_stop_words(train_data+test_data+dev_data, 1000, vocab, 100, True)
    # stopwords = load_pickle('pickle/mov_stopwords.pickle')

    # train_k, train_v = load_kv_dataset(train_data, kv_pairs, stopwords)
    # save_pickle(train_k, 'pickle/mov_train_k.pickle')
    # save_pickle(train_v, 'pickle/mov_train_v.pickle')
    # test_k, test_v = load_kv_dataset(test_data, kv_pairs, stopwords)
    # save_pickle(test_k, 'pickle/mov_test_k.pickle')
    # save_pickle(test_v, 'pickle/mov_test_v.pickle')
    # dev_k, dev_v = load_kv_dataset(dev_data, kv_pairs, stopwords)
    # save_pickle(test_k, 'pickle/mov_dev_k.pickle')
    # save_pickle(test_v, 'pickle/mov_dev_v.pickle')
    


In [2]:
# key-value memories: preprocess, key-value inverted index, embedding
import os
import re # regex
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora

def read1k():
    return f.read(1024)

def process_data(chunk, text):
    text.append(str(chunk)) # 'utf8' codec can't decode byte 0xc3

def rmsword(corpus, stopwords): # remove stopwords from corpus
    i = 0
    for elem in corpus:
        for sword in stopwords:
            if elem == sword:
                while True:
                    try:
                        corpus.remove(elem) # this throws an error if elem not in corpus (might have been removed already)
                        i += 1
                    except:
                        break
    return i # returns number of stopwords removed

def chunks(l, n): # Yield successive n-sized chunks from list l
    for i in range(0, len(l), n):
        yield l[i:i + n] # returns a generator

def chunksep(l, s): # Yield successive chunks from list l separated by s
    g = []
    for el in l:
        if el == s:
            yield g
            g = []
        g.append(el)
    yield g
    
def lsttoint(lst): # todo: check for numeric char
    res = ''
    for e in lst:
        res += str(e)
    return int(res)

def docstr2lst(text): # text string with sentences separated by ., returns list of lists of sentences
    memraw = []
    for mem in text.split('.'):
        memraw.append(mem.split(' '))
    return memraw

def docstr2lstpre(text): # text string with sentences separated by ., returns list of lists of sentences & preprocess
    memraw = []
    for mem in text.split('.'):
        thismem = preprocess_string(mem)
        if len(thismem) > 0:
            memraw.append(preprocess_string(mem))
    return memraw

def text2bow(memraw, memdict): # raw text to bow (maps each token to its id, takes a list of lists of sentence words)
    membow = []
    for mem in memraw:
        memline = []
        for memw in mem:
            memline.append(memdict.token2id[memw])
        membow.append(memline)
    return membow

def windowlvlstr(text, lenW): # return key=entire window, value=center word, string version
    if lenW % 2 == 0 or len(text) <= lenW:
        return ([],[])
    textl = text.split(' ')
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(textl)-lenW2):
        thiskey = []
        retvals.append(textl[ictr])
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(textl[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

def windowlvl(text, lenW): # return key=entire window, value=center word, BOW version (text a list of ids)
    if lenW % 2 == 0 or len(text) < lenW:
        return ([],[])
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(text)-lenW2):
        thiskey = []
        retvals.append(text[ictr]) # center encoding: would need to add a different dict here
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(text[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

# for window+center encoding: add a step for the center word:
# translate the center back to the original with dic 0, then with dic 1 to the center encoding
def windowclvl(text, lenW, memdict, cdict): # return key=entire window, value=center word, BOW version with center
    if lenW % 2 == 0 or len(text) < lenW:
        return ([],[])
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(text)-lenW2):
        thiskey = []
        retvals.append(cdict.token2id[memdict[text[ictr]]]) # center encoding: different dict for ctr word
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(text[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

if __name__=="__main__":    
    # get document data
    text = 'this is a test with a lot of ambiguity. docs are split by periods.'
    # preprocess and build dictionary
    memraw = docstr2lstpre(text)
    print(memraw)
    memdict = corpora.Dictionary(memraw)
    print(memdict.token2id)    
    # text to bow
    membow = text2bow(memraw, memdict)
    print(membow)
    # feature mapping and build inverted index
    kvinvind = {}
    for imem in range(len(membow)):
        keyvalbow = windowlvl(membow[imem], 3)
        print(keyvalbow)
        for ikvb in range(len(keyvalbow[0])):
            kvinvind[hash(lsttoint(keyvalbow[0][ikvb]))] = keyvalbow[1][0]
    print(kvinvind)

[['test', 'lot', 'ambigu'], ['doc', 'split', 'period']]
{'ambigu': 0, 'lot': 1, 'test': 2, 'doc': 3, 'period': 4, 'split': 5}
[[2, 1, 0], [3, 5, 4]]
([[2, 1, 0]], [1])
([[3, 5, 4]], [5])
{210: 1, 354: 5}


In [40]:
# key-value memory network
import os
import math
import re # regex
import numpy as np
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora

def read1k():
    return f.read(1024)

def process_data(chunk, text):
    text.append(str(chunk)) # 'utf8' codec can't decode byte 0xc3

def rmsword(corpus, stopwords): # remove stopwords from corpus
    i = 0
    for elem in corpus:
        for sword in stopwords:
            if elem == sword:
                while True:
                    try:
                        corpus.remove(elem) # this throws an error if elem not in corpus (might have been removed already)
                        i += 1
                    except:
                        break
    return i # returns number of stopwords removed

def chunks(l, n): # Yield successive n-sized chunks from list l
    for i in range(0, len(l), n):
        yield l[i:i + n] # returns a generator

def chunksep(l, s): # Yield successive chunks from list l separated by s
    g = []
    for el in l:
        if el == s:
            yield g
            g = []
        g.append(el)
    yield g
    
def lsttoint(lst): # todo: check for numeric char
    res = ''
    for e in lst:
        res += str(e)
    return int(res)

def docstr2lst(text): # text string with sentences separated by ., returns list of lists of sentences
    memraw = []
    for mem in text.split('.'):
        memraw.append(mem.split(' '))
    return memraw

def docstr2lstpre(text): # text string with sentences separated by ., returns list of lists of sentences & preprocess
    memraw = []
    for mem in text.split('.'):
        thismem = preprocess_string(mem)# todo: check preprocessing, maybe just stem
        if len(thismem) > 0:
            memraw.append(preprocess_string(mem))
    return memraw

def text2bow(memraw, memdict): # raw text to bow (maps each token to its id, takes a list of lists of sentence words)
    membow = []
    for mem in memraw:
        memline = []
        for memw in mem:
            memline.append(memdict.token2id[memw])
        membow.append(memline)
    return membow

def windowlvlstr(text, lenW): # return key=entire window, value=center word, string version
    if lenW % 2 == 0 or len(text) <= lenW:
        return ([],[])
    textl = text.split(' ')
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(textl)-lenW2):
        thiskey = []
        retvals.append(textl[ictr])
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(textl[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

def windowlvl(text, lenW): # return key=entire window, value=center word, BOW version (text a list of ids)
    if lenW % 2 == 0 or len(text) < lenW:
        return ([],[])
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(text)-lenW2):
        thiskey = []
        retvals.append(text[ictr]) # center encoding: would need to add a different dict here
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(text[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

# for window+center encoding: add a step for the center word:
# translate the center back to the original with dic 0, then with dic 1 to the center encoding
def windowclvl(text, lenW, memdict, cdict): # return key=entire window, value=center word, BOW version with center
    if lenW % 2 == 0 or len(text) < lenW:
        return ([],[])
    retkeys = []
    retvals = []
    lenW2 = int((lenW-1)/2)
    for ictr in range(lenW2, len(text)-lenW2):
        thiskey = []
        retvals.append(cdict.token2id[memdict[text[ictr]]]) # center encoding: different dict for ctr word
        for ikey in range(ictr-lenW2, ictr+lenW2+1):
            thiskey.append(text[ikey])
        retkeys.append(thiskey)
    return (retkeys, retvals)

def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

def lj(j, J, k, d): 
    return (1-j/J)-(k/d)*(1-2*j/J)

def embed(facts_raw, query_raw, uniq):
    facts = facts_raw.lower().split('.')
    facts.append(query_raw.lower())
    #senttoken = [ [word for word in sentence.lower().split(' ') if word not in stoplist] for sentence in document ]
    senttoken = []
    for idoc in range(len(facts)):
        thissen = facts[idoc].lower().split(' ')
        tokendoc = []
        for idx, thiswrd in enumerate(thissen):
            if uniq:
                tokendoc.append(thiswrd + '_' + str(idoc)) # for seaparate encoding (Jonathan Huis blog)
            else:
                tokendoc.append(thiswrd)
        senttoken.append(tokendoc)
    dictionary = corpora.Dictionary(senttoken)
    facts.pop(len(facts)-1) # query at the end of document
    d_embed = len(dictionary) # embedding dimension
    return(senttoken, facts, dictionary, d_embed)

def preinvind(facts_raw, query_raw, wsize): # preprocess and build inverted index
    # preprocess and build dictionary
    memraw = docstr2lstpre(facts_raw + query_raw)
    print(memraw)
    memdict = corpora.Dictionary(memraw) 
    # remove query from end
    query = memraw.pop(len(memraw)-1)
    # text to bow
    membow = text2bow(memraw, memdict)
    # feature mapping and build inverted index
    kvinvind = {}
    for imem in range(len(membow)):
        keyvalbow = windowlvl(membow[imem], wsize)
        for ikvb in range(len(keyvalbow[0])):
            kvinvind[hash(lsttoint(keyvalbow[0][ikvb]))] = keyvalbow[1][0]
    return memdict, kvinvind, query

if __name__=="__main__":
    # load input data
    #data = open('input1.txt', 'r').read() # input text file: sentences separated by .

    # hyperparameters
    lr = 1.3 # learning rate
    
    # input to memory embedding mi = A * xi + tA
        #stoplist = set('for a of the and to in'.split())
    stoplist = []
    facts_raw = 'Mary moved to the bathroom. John went to the hallway.'
    
    # create key-value pairs
    query_raw = 'Where is Mary'
    
    dictionary, kvinvind, query = preinvind(facts_raw, query_raw, 3) # key-value inv idx
    d_embed = len(dictionary)
    
    #senttoken, facts, dictionary, d_embed = embed(facts_raw, query_raw, False)
    voc = 1 # todo: check
    n_memories = len(kvinvind)
    
    print(kvinvind)
    print(query)

    # initiate weigth matrices
    A = np.random.randn(d_embed, voc)*0.01 # input to memory embedding = key-value and query embeddings
    B = np.random.randn(d_embed, voc)*0.01 # query embedding = for y-embeddings
    R1 = np.random.randn(voc, d_embed)*0.01 # final weight matrix = R1 for hop-to-hop query updates
    
    # memory for Adagrad
    mA = np.zeros_like(A)
    mB = np.zeros_like(B)
    mR1 = np.zeros_like(R1)

    phi_k = np.zeros((n_memories, voc)) # keys
    phi_v = np.zeros((n_memories, voc)) # values
    phi_y = np.zeros(voc) # candidates
    i = 0
    for ik in kvinvind.keys():
        phi_k[i] = ik
        phi_v[i] = ik # for now the same as keys
        i += 1
    
    for i in range(voc):
        phi_y[i] = i # each word in the vocabulary is a potential candidate
        
    # todo: key hashing (inverted index)
         
    for iterctr in range(30):

        # forward pass
        
        # embedding key = A * phi_k
        key = np.zeros((n_memories, d_embed))
        for i in range(n_memories):
            key[i] = np.dot(A, phi_k[i].T) # simple embedding
            #m[i][j] = lj(j,len(document[i]),j,d_embed) * A[i][j] * x[i][j] # with positional encoding

        # embedding query q = A * phi_x
        phi_x = np.zeros(voc)
        qj1 = np.zeros((d_embed,1))
        for j in range(len(query)):
            phi_x[j] = dictionary.token2id[query[j]]
        qj1 = np.dot(A, phi_x)
        
        # key addressing

        # read head of the Neural Turing Machine
        # match of query with key p = softmax(qj1 * A * key) for all i
        pk = np.zeros((n_memories, d_embed))
        pk = softmax(np.dot(qj1, key.T))

        # output corresponding to input xi: ci = C * xi
        val = np.zeros((n_memories, d_embed))
        for i in range(n_memories):
            val[i] = np.dot(A, phi_v[i].T)
            
        # value reading

        # internal state of the controller
        # response vector from memory o = sum pi * ci
        o = np.zeros(d_embed)
        o = np.dot(pk.T, val)
        
        # 2nd hop
        #qj1 = np.dot(R1, (qj1 + o))
        #pk = softmax(np.dot(qj1, key.T))
        #o = np.dot(pk.T, val)
        #qj1 = np.dot(R1, (qj1 + o))
        
        # 3rd hop
        #qj1 = np.dot(R1, (qj1 + o))
        #pk = softmax(np.dot(qj1, key.T))
        #o = np.dot(pk.T, val)
        #qj1 = np.dot(R1, (qj1 + o))

        # predicted label a = softmax( R1 * (o + u) * B * phi_y)
        #a_predict = softmax(np.dot(R1, (qj1 + o)))
        a_predict = softmax(qj1 * np.dot(B, phi_y))
        #a_predict = softmax(qj1 * np.dot(A, phi_y)) # only using A
        #print(a_predict)

        # backpropagation

        dA = np.zeros_like(A)
        dB = np.zeros_like(B)
        dR1 = np.zeros_like(R1)

        truth = np.zeros(voc)
        truth[0] = 1. # answer (bathroom)
        dy = a_predict - truth
        #print(dy)
        #print('V: %d' % (voc))
        #print('d: %d', (d_embed))
        ABunit = np.pad(np.identity(voc), ((0,d_embed-voc),(0,0)), 'constant', constant_values=(0))
        R1unit = np.pad(np.identity(voc), ((0,0), (0,d_embed-voc)), 'constant', constant_values=(0))

        # dA = dy a_predict * (1-a_predict) R1 (phi_x + sumi ( p[i] (1-p[i]) ( phi_x A phi_K + A phi_x phi_K ) A phi_V + pki phi_V)
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += pk[i]*(1.-pk[i]) * np.dot(np.dot(qj1, np.dot(ABunit, phi_k[i].T)), val[i])
        dEAtmp = R1 * dEAtmp
        dA = (np.dot(dy, a_predict * (1-a_predict)) * dEAtmp).T
        #print(dA)

        # dB = dy a_predict * (1-a_predict) q phi_y
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += pk[i]*(1.-pk[i])*np.dot(np.dot(ABunit, qj1), phi_y[i])
        dEAtmp = R1 * dEAtmp
        dB = (np.dot(dy, a_predict*(1-a_predict)) * dEAtmp).T
        #print(dB)

        # dR1 = dy a_predict * (1-a_predict) (q + o) B phi_y
        #print(np.shape(np.dot((qj1 + o), np.dot(B, phi_y.T))))
        dR1 = np.dot(dy, a_predict*(1-a_predict)) * R1unit * np.dot((qj1 + o), np.dot(B, phi_y.T))
        #print(dR1)   

        # maybe clip ?
        #for dweights in [dA,dB,dR1]:
            #np.clip(dweights, -5., 5., out=dweights) # exploding gradients (but seems well-behaved enough)

        # update weights with Adagrad
        for weights, dweights, memwghts in zip([A,B,R1], [dA,dB,dR1], [mA,mB,mR1]):
            #memwghts += dweights * dweights
            #weights += -lr * dweights / np.sqrt(memwghts + 1.e-8)
            weights += -lr * dweights

        #print(A)
    print(a_predict)
    #print(np.argmax(a_predict))
    print(dictionary[np.argmax(a_predict)])

[['mari', 'move', 'bathroom'], ['john', 'went', 'hallwai'], ['mari']]
{120: 2, 453: 5}
['mari']


ValueError: shapes (6,1) and (6,) not aligned: 1 (dim 1) != 6 (dim 0)

In [49]:
# neural turing machine
import numpy as np
import math

def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

def simK(u, v): # similarity measure
    return np.dot(u, v) / np.linalg.norm(u) / np.linalg.norm(v)

if __name__=="__main__":
    # setup
    Mt = np.array([[1, 0, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1]]) # memory (N slots), M=3
    wt = np.array([0.25, 0.25, 0.25, 0.25]) # weights 1-N: one for each memory entry
    wt_1 = wt # weights from previous time step
    
    # weights
    beta = 0.1 # key strength
    kt = np.array([0.5, 0.2, 0.1]) # key vector
    wt = softmax(beta * simK(Mt, kt)) # content addressing
    print(wt)
    gt = 0.35 # interpolation gate
    wt = gt * wt + (1 - gt) * wt_1 # gated weighing
    print(wt)
    st = np.array([0.1, 0.8, 0.1]) # shift weights
    wt = np.convolve(wt, st, mode='same') # convolutional shift
    print(wt)
    gamma = 2.5
    wt = np.power(wt, gamma) / np.sum(np.power(wt, gamma)) # sharpening
    print(wt)
    
    # read
    rt = np.dot(wt, Mt) # dim M, weighted memories

    # write
    et = np.array([1, 0, 0, 0]) # erase vector
    at = np.array([0, 1, 0, 0]) # add vector
    Mt = np.dot(Mt, (1 - np.dot(wt, et))) # erase
    Mt = Mt + np.dot(wt, at) # add
    print(Mt)

    # fill memory


    # move heand to start


    # while not at memory end


    # receive input vector


    # write input to head


    # increment head position by 1


    # return head to start location


    # while true


    # read output vector from head location


    # emit output


    # increment head by 1

[0.25079645 0.24598652 0.25242057 0.25079645]
[0.25027876 0.24859528 0.2508472  0.25027876]
[0.22508253 0.24898882 0.25056516 0.22530773]
[0.21752991 0.27997169 0.28442399 0.21807441]
[[1.06244178 0.27997169 1.06244178]
 [0.27997169 1.06244178 1.06244178]
 [1.06244178 1.06244178 0.27997169]
 [1.06244178 0.27997169 1.06244178]]


In [31]:
# key-value-query attention test (example from Jay Alammars blog)
import numpy as np
import math

def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

if __name__=="__main__":
    input_sent = 'thinking machines'
    # embeddings
    x1 = np.array([1, 0, 0, 0])
    x2 = np.array([0, 1, 0, 0])

    # weights
    Wk = np.array([[-1, 0, 1], [0, 1, 1], [1, -1, 0], [1, 0, -1]])
    Wv = np.array([[0, 1, -1], [-1, 0, 1], [1, 0, 1], [0, 1, 1]])
    Wq = np.array([[-1, 1, 0], [1, 1, 0], [-1, 0, 1], [-1, 1, 0]])

    # query, key, value
    q1 = np.dot(Wq.T, x1)
    k1 = np.dot(Wk.T, x1)
    v1 = np.dot(Wv.T, x1)

    q2 = np.dot(Wq.T, x2)
    k2 = np.dot(Wk.T, x2)
    v2 = np.dot(Wv.T, x2)

    print('qkv1 ', q1, k1, v1)
    print('qkv2 ', q2, k2, v2)

    # scores
    s1 = np.dot(q1, k1)
    s2 = np.dot(q2, k2)
    print('scores ', s1, s2)

    # key dimension
    d1 = np.shape(k1)[0]
    d2 = np.shape(k2)[0]
    s1 /= math.sqrt(d1)
    s2 /= math.sqrt(d2)
    print('scores corrected for dim ', s1, s2)
    
    # softmax
    s1 = softmax(s1)
    s2 = softmax(s2)
    print('softmax ', s1, s2)
    
    # weighted value vectors
    z1 = s1 * v1
    z2 = s2 * v2
    print('weighted values ', z1, z2)
    
    # sum up weighted value vectors
    zsum = z1 + z2
    print('weighted sum ', zsum)

qkv1  [-1  1  0] [-1  0  1] [ 0  1 -1]
qkv2  [1 1 0] [0 1 1] [-1  0  1]
scores  1 1
scores corrected for dim  0.5773502691896258 0.5773502691896258
softmax  1.0 1.0
weighted values  [ 0.  1. -1.] [-1.  0.  1.]
weighted sum  [-1.  1.  0.]


In [32]:
# key-value-query attention test - full matrix version
import numpy as np
import math

def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

if __name__=="__main__":
    input_sent = 'thinking machines'
    # embeddings
    xn = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])

    # weights
    Wk = np.array([[-1, 0, 1], [0, 1, 1], [1, -1, 0], [1, 0, -1]])
    Wv = np.array([[0, 1, -1], [-1, 0, 1], [1, 0, 1], [0, 1, 1]])
    Wq = np.array([[-1, 1, 0], [1, 1, 0], [-1, 0, 1], [-1, 1, 0]])

    # query, key, value
    qn = np.dot(Wq.T, xn.T)
    kn = np.dot(Wk.T, xn.T)
    vn = np.dot(Wv.T, xn.T)

    print('qkvn ', qn, kn, vn)

    # scores
    sn = np.dot(qn.T, kn)
    print('scores ', sn)

    # key dimension
    dn = np.shape(kn)[0]
    print(dn)
    sn = np.true_divide(sn, np.sqrt(dn))
    print('scores corrected for dim ', sn)
    
    # softmax
    sn = softmax(sn)
    print('softmax ', sn)
    
    # weighted value vectors
    zn = np.dot(sn, vn.T)
    print('weighted values ', zn)
    
    # sum up weighted value vectors
    zsum = np.sum(zn, axis=0)
    print('weighted sum ', zsum)

qkvn  [[-1  1]
 [ 1  1]
 [ 0  0]] [[-1  0]
 [ 0  1]
 [ 1  1]] [[ 0 -1]
 [ 1  0]
 [-1  1]]
scores  [[ 1  1]
 [-1  1]]
3
scores corrected for dim  [[ 0.57735027  0.57735027]
 [-0.57735027  0.57735027]]
softmax  [[0.3016453  0.3016453 ]
 [0.09506409 0.3016453 ]]
weighted values  [[-0.3016453   0.3016453   0.        ]
 [-0.3016453   0.09506409  0.20658121]]
weighted sum  [-0.60329061  0.39670939  0.20658121]
